In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 126332 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u442-b06~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u442-b06~us1

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
from tqdm import tqdm_notebook
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [5]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Medicare Part D Prescribers

https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers

In [6]:
claims = sqlContext.read.csv('/content/drive/MyDrive/ISYE 6740 Project Files/Medicare Part D Prescribers - by Provider and Drug/2022/MUP_DPR_RY24_P04_V10_DY22_NPIBN.csv',header = True, inferSchema = True)
claims.head()

Row(Prscrbr_NPI=1003000126, Prscrbr_Last_Org_Name='Enkeshafi', Prscrbr_First_Name='Ardalan', Prscrbr_City='Bethesda', Prscrbr_State_Abrvtn='MD', Prscrbr_State_FIPS='24', Prscrbr_Type='Internal Medicine', Prscrbr_Type_Src='Claim-Specialty', Brnd_Name='Amlodipine Besylate', Gnrc_Name='Amlodipine Besylate', Tot_Clms=19, Tot_30day_Fills=41.0, Tot_Day_Suply=1230, Tot_Drug_Cst=246.17, Tot_Benes=14, GE65_Sprsn_Flag=None, GE65_Tot_Clms=19, GE65_Tot_30day_Fills=41.0, GE65_Tot_Drug_Cst=246.17, GE65_Tot_Day_Suply=1230, GE65_Bene_Sprsn_Flag=None, GE65_Tot_Benes=14)

In [7]:
claims.columns

['Prscrbr_NPI',
 'Prscrbr_Last_Org_Name',
 'Prscrbr_First_Name',
 'Prscrbr_City',
 'Prscrbr_State_Abrvtn',
 'Prscrbr_State_FIPS',
 'Prscrbr_Type',
 'Prscrbr_Type_Src',
 'Brnd_Name',
 'Gnrc_Name',
 'Tot_Clms',
 'Tot_30day_Fills',
 'Tot_Day_Suply',
 'Tot_Drug_Cst',
 'Tot_Benes',
 'GE65_Sprsn_Flag',
 'GE65_Tot_Clms',
 'GE65_Tot_30day_Fills',
 'GE65_Tot_Drug_Cst',
 'GE65_Tot_Day_Suply',
 'GE65_Bene_Sprsn_Flag',
 'GE65_Tot_Benes']

In [ ]:
claims.printSchema()

root
 |-- Prscrbr_NPI: integer (nullable = true)
 |-- Prscrbr_Last_Org_Name: string (nullable = true)
 |-- Prscrbr_First_Name: string (nullable = true)
 |-- Prscrbr_City: string (nullable = true)
 |-- Prscrbr_State_Abrvtn: string (nullable = true)
 |-- Prscrbr_State_FIPS: string (nullable = true)
 |-- Prscrbr_Type: string (nullable = true)
 |-- Prscrbr_Type_Src: string (nullable = true)
 |-- Brnd_Name: string (nullable = true)
 |-- Gnrc_Name: string (nullable = true)
 |-- Tot_Clms: integer (nullable = true)
 |-- Tot_30day_Fills: double (nullable = true)
 |-- Tot_Day_Suply: integer (nullable = true)
 |-- Tot_Drug_Cst: double (nullable = true)
 |-- Tot_Benes: integer (nullable = true)
 |-- GE65_Sprsn_Flag: string (nullable = true)
 |-- GE65_Tot_Clms: integer (nullable = true)
 |-- GE65_Tot_30day_Fills: double (nullable = true)
 |-- GE65_Tot_Drug_Cst: double (nullable = true)
 |-- GE65_Tot_Day_Suply: integer (nullable = true)
 |-- GE65_Bene_Sprsn_Flag: string (nullable = true)
 |-- GE65_T

In [ ]:
# Get non-null counts
non_null_counts = claims.select([count(col(c)).alias(c) for c in claims.columns])
non_null_dict = non_null_counts.collect()[0].asDict()

# Get distinct counts
distinct_counts = claims.select([countDistinct(col(c)).alias(c) for c in claims.columns])
distinct_dict = distinct_counts.collect()[0].asDict()

# Combine both into a list of tuples
combined_data = [
    (col_name, non_null_dict.get(col_name, 0), distinct_dict.get(col_name, 0))
    for col_name in claims.columns
]

# Create final DataFrame
transpose_df = spark.createDataFrame(
    combined_data,
    ["Column", "Non-Null Count", "Distinct Count"]
)

transpose_df.show(30)
print("Total number of rows in the dataset:", claims.count())

+--------------------+--------------+--------------+
|              Column|Non-Null Count|Distinct Count|
+--------------------+--------------+--------------+
|         Prscrbr_NPI|      25869521|       1057566|
|Prscrbr_Last_Org_...|      25869521|        251689|
|  Prscrbr_First_Name|      25869521|         83537|
|        Prscrbr_City|      25869521|         12193|
|Prscrbr_State_Abrvtn|      25869521|            61|
|  Prscrbr_State_FIPS|      25869521|            61|
|        Prscrbr_Type|      25869516|           180|
|    Prscrbr_Type_Src|      25869521|             3|
|           Brnd_Name|      25869521|          3021|
|           Gnrc_Name|      25869521|          1757|
|            Tot_Clms|      25869521|          3752|
|     Tot_30day_Fills|      25869521|         31834|
|       Tot_Day_Suply|      25869521|         74560|
|        Tot_Drug_Cst|      25869521|       2962959|
|           Tot_Benes|      11362619|          1354|
|     GE65_Sprsn_Flag|      11625697|         

In [ ]:
from pyspark.sql.functions import countDistinct

# Count distinct states per NPI
state_counts = claims.groupBy("Prscrbr_NPI").agg(countDistinct("Prscrbr_State_Abrvtn").alias("num_states"))

# Filter where more than one state exists
npis_multiple_states = state_counts.filter("num_states > 1")

npis_multiple_states.show(truncate=False)

+-----------+----------+
|Prscrbr_NPI|num_states|
+-----------+----------+
+-----------+----------+



In [ ]:
# Count distinct cities per NPI
city_counts = claims.groupBy("Prscrbr_NPI").agg(countDistinct("Prscrbr_City").alias("num_cities"))

# Filter where more than one city exists
npis_multiple_cities = city_counts.filter("num_cities > 1")

npis_multiple_cities.show(truncate=False)

+-----------+----------+
|Prscrbr_NPI|num_cities|
+-----------+----------+
+-----------+----------+



In [ ]:
# Count distinct types per NPI
type_counts = claims.groupBy("Prscrbr_NPI").agg(countDistinct("Prscrbr_Type").alias("num_types"))

# Filter where more than one city exists
npis_multiple_types = type_counts.filter("num_types > 1")

npis_multiple_types.show(truncate=False)

+-----------+---------+
|Prscrbr_NPI|num_types|
+-----------+---------+
+-----------+---------+



## Payment data

In [8]:
payment = sqlContext.read.csv('/content/drive/MyDrive/ISYE 6740 Project Files/PGYR2022_P01302025_01212025 (1)/OP_DTL_GNRL_PGYR2022_P01302025_01212025.csv',header = True, inferSchema = True)
payment.head()

Row(Change_Type='CHANGED', Covered_Recipient_Type='Covered Recipient Teaching Hospital', Teaching_Hospital_CCN='220162', Teaching_Hospital_ID='9984', Teaching_Hospital_Name='Allegheny General Hospital', Covered_Recipient_Profile_ID=None, Covered_Recipient_NPI=None, Covered_Recipient_First_Name=None, Covered_Recipient_Middle_Name=None, Covered_Recipient_Last_Name=None, Covered_Recipient_Name_Suffix=None, Recipient_Primary_Business_Street_Address_Line1='320 East North Avenue', Recipient_Primary_Business_Street_Address_Line2=None, Recipient_City='Pittsburgh', Recipient_State='PA', Recipient_Zip_Code='15212', Recipient_Country='United States', Recipient_Province=None, Recipient_Postal_Code=None, Covered_Recipient_Primary_Type_1=None, Covered_Recipient_Primary_Type_2=None, Covered_Recipient_Primary_Type_3=None, Covered_Recipient_Primary_Type_4=None, Covered_Recipient_Primary_Type_5=None, Covered_Recipient_Primary_Type_6=None, Covered_Recipient_Specialty_1=None, Covered_Recipient_Specialty_2

In [ ]:
# Get non-null counts
non_null_counts = payment.select([count(col(c)).alias(c) for c in payment.columns])
non_null_dict = non_null_counts.collect()[0].asDict()

# Get distinct counts
distinct_counts = payment.select([countDistinct(col(c)).alias(c) for c in payment.columns])
distinct_dict = distinct_counts.collect()[0].asDict()

# Combine both into a list of tuples
combined_data = [
    (col_name, non_null_dict.get(col_name, 0), distinct_dict.get(col_name, 0))
    for col_name in payment.columns
]

# Create final DataFrame
transpose_df = spark.createDataFrame(
    combined_data,
    ["Column", "Non-Null Count", "Distinct Count"]
)

In [ ]:
transpose_df.show(100, truncate=False)
print("Total number of rows in the dataset:", payment.count())

+-----------------------------------------------------------------+--------------+--------------+
|Column                                                           |Non-Null Count|Distinct Count|
+-----------------------------------------------------------------+--------------+--------------+
|Change_Type                                                      |13286449      |240           |
|Covered_Recipient_Type                                           |13286298      |49            |
|Teaching_Hospital_CCN                                            |39056         |1354          |
|Teaching_Hospital_ID                                             |39056         |1297          |
|Teaching_Hospital_Name                                           |39056         |2065          |
|Covered_Recipient_Profile_ID                                     |13247361      |864496        |
|Covered_Recipient_NPI                                            |13234868      |859808        |
|Covered_Recipient_F

In [ ]:
payment.select("Form_of_Payment_or_Transfer_of_Value").distinct().rdd.flatMap(lambda x: x).collect()

['Stock, stock option, or any other ownership interest',
 'Dividend, profit or other return on investment',
 'Cash or cash equivalent',
 'In-kind items and services',
 'Stock option',
 'Stock',
 '08/11/2022',
 '02/22/2022',
 None]

In [ ]:
payment.groupBy("Form_of_Payment_or_Transfer_of_Value") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+----------------------------------------------------+--------+
|Form_of_Payment_or_Transfer_of_Value                |count   |
+----------------------------------------------------+--------+
|In-kind items and services                          |11445399|
|Cash or cash equivalent                             |1839660 |
|Stock option                                        |543     |
|NULL                                                |388     |
|Dividend, profit or other return on investment      |220     |
|Stock, stock option, or any other ownership interest|173     |
|Stock                                               |64      |
|08/11/2022                                          |1       |
|02/22/2022                                          |1       |
+----------------------------------------------------+--------+



In [ ]:
payment.select("Nature_of_Payment_or_Transfer_of_Value").distinct().rdd.flatMap(lambda x: x).collect()

['Education',
 'Entertainment',
 'Travel and Lodging',
 'Compensation for serving as faculty or as a speaker for a medical education program',
 'Acquisitions',
 'Debt forgiveness',
 'Charitable Contribution',
 'Current or prospective ownership or investment interest',
 'Honoraria',
 'Grant',
 'Royalty or License',
 'Food and Beverage',
 'Space rental or facility fees (teaching hospital only)',
 'Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program',
 'Consulting Fee',
 'Gift',
 'Long term medical supply or device loan',
 '1',
 None]

In [ ]:
payment.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |12126695|
|Travel and Lodging                                                                                                                                |472240  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|232452  |
|Consulting Fee                                     

In [9]:
selected_cols = [
    "Covered_Recipient_NPI", "Total_Amount_of_Payment_USDollars", 'Number_of_Payments_Included_in_Total_Amount',
    'Physician_Ownership_Indicator','Third_Party_Payment_Recipient_Indicator','Charity_Indicator','Covered_or_Noncovered_Indicator_1'
]

payment_selected = payment.select(*selected_cols)

In [ ]:
def dups(by_col,view_col, df):
  type_counts = df.groupBy(by_col).agg(countDistinct(view_col).alias(view_col+"_types"))

  # Filter where more than one city exists
  multiple_types = type_counts.filter(view_col+"_types"+" > 1")

  multiple_types.show(truncate=False)

In [ ]:
from pyspark.sql.functions import countDistinct, col

def count_npi_by_ownership_variation(by_col,view_col,df):
    # Step 1: Count distinct Physician_Ownership_Indicator values per NPI
    npi_ownership_counts = df.groupBy(by_col).agg(
        countDistinct(view_col).alias("distinct_ownership_values")
    )

    # Step 2: Classify NPIs by variation type (1 = only one value, >1 = multiple)
    ownership_variation_counts = npi_ownership_counts.groupBy("distinct_ownership_values").count().orderBy("distinct_ownership_values")

    # Step 3: Show results
    ownership_variation_counts.show(truncate=False)

    # Optional: Return DataFrame for further use
    return ownership_variation_counts


In [ ]:
dups("Covered_Recipient_NPI","Physician_Ownership_Indicator",payment_selected)

+---------------------+-----------------------------------+
|Covered_Recipient_NPI|Physician_Ownership_Indicator_types|
+---------------------+-----------------------------------+
|1780644211           |2                                  |
|1629495502           |2                                  |
|1114104999           |2                                  |
|1023110103           |2                                  |
|1740289701           |2                                  |
|1326242991           |2                                  |
|1336434182           |2                                  |
|1841230976           |2                                  |
|1467489542           |2                                  |
|1326026287           |2                                  |
|1154598068           |2                                  |
|1043258270           |2                                  |
|1851345433           |2                                  |
|1700849411           |2                

In [ ]:
count_npi_by_ownership_variation("Covered_Recipient_NPI","Physician_Ownership_Indicator",payment_selected)

+-------------------------+------+
|distinct_ownership_values|count |
+-------------------------+------+
|0                        |55164 |
|1                        |785712|
|2                        |18933 |
+-------------------------+------+



DataFrame[distinct_ownership_values: bigint, count: bigint]

In [ ]:
dups("Covered_Recipient_NPI","Third_Party_Payment_Recipient_Indicator",payment_selected)

+---------------------+---------------------------------------------+
|Covered_Recipient_NPI|Third_Party_Payment_Recipient_Indicator_types|
+---------------------+---------------------------------------------+
|1962705590           |2                                            |
|1043436819           |2                                            |
|1992064026           |2                                            |
|1518490234           |2                                            |
|1043425036           |2                                            |
|1467710889           |2                                            |
|1023110103           |2                                            |
|1477519247           |2                                            |
|1578094470           |2                                            |
|1487673208           |2                                            |
|1851345433           |2                                            |
|1760447643         

In [ ]:
count_npi_by_ownership_variation("Covered_Recipient_NPI","Third_Party_Payment_Recipient_Indicator",payment_selected)

+-------------------------+------+
|distinct_ownership_values|count |
+-------------------------+------+
|0                        |7     |
|1                        |840302|
|2                        |18761 |
|3                        |739   |
+-------------------------+------+



DataFrame[distinct_ownership_values: bigint, count: bigint]

In [ ]:
dups("Covered_Recipient_NPI","Charity_Indicator",payment_selected)

+---------------------+-----------------------+
|Covered_Recipient_NPI|Charity_Indicator_types|
+---------------------+-----------------------+
|1841419348           |2                      |
|1508154428           |2                      |
|1477579647           |2                      |
|1295736916           |2                      |
|1346339652           |2                      |
|1841318284           |2                      |
|1073624102           |2                      |
|1689606832           |2                      |
|1942213343           |2                      |
|1720318355           |2                      |
|1225474034           |2                      |
|1679675995           |2                      |
|1912902651           |2                      |
|1134114382           |2                      |
|1487691127           |2                      |
|1184919524           |2                      |
|1326304890           |2                      |
|1598715914           |2                

In [ ]:
count_npi_by_ownership_variation("Covered_Recipient_NPI","Charity_Indicator",payment_selected)

+-------------------------+------+
|distinct_ownership_values|count |
+-------------------------+------+
|0                        |402123|
|1                        |457452|
|2                        |234   |
+-------------------------+------+



DataFrame[distinct_ownership_values: bigint, count: bigint]

In [ ]:
dups("Covered_Recipient_NPI","Covered_or_Noncovered_Indicator_1",payment_selected)

+---------------------+---------------------------------------+
|Covered_Recipient_NPI|Covered_or_Noncovered_Indicator_1_types|
+---------------------+---------------------------------------+
|1750333050           |2                                      |
|1346521945           |2                                      |
|1588005128           |2                                      |
|1922283506           |2                                      |
|1144644287           |2                                      |
|1881850261           |2                                      |
|1164515318           |2                                      |
|1679524268           |2                                      |
|1477575546           |2                                      |
|1881766962           |2                                      |
|1316988165           |2                                      |
|1598701468           |2                                      |
|1962448712           |2                

In [ ]:
payment_selected.groupBy("Covered_or_Noncovered_Indicator_1") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+---------------------------------+--------+
|Covered_or_Noncovered_Indicator_1|count   |
+---------------------------------+--------+
|Covered                          |12227918|
|NULL                             |714715  |
|Non-Covered                      |343811  |
|No                               |5       |
+---------------------------------+--------+



In [12]:
from pyspark.sql.functions import when

payment_selected = payment_selected.withColumn(
    "Is_Covered",
    when(payment_selected["Covered_or_Noncovered_Indicator_1"] == "Covered", 1).otherwise(0)
)

payment_selected.groupBy("Is_Covered") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+----------+--------+
|Is_Covered|count   |
+----------+--------+
|1         |12227918|
|0         |1058531 |
+----------+--------+



In [13]:
payment_selected.groupBy("Third_Party_Payment_Recipient_Indicator") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+---------------------------------------+--------+
|Third_Party_Payment_Recipient_Indicator|count   |
+---------------------------------------+--------+
|No Third Party Payment                 |13185006|
|Entity                                 |85941   |
|Individual                             |15112   |
|NULL                                   |390     |
+---------------------------------------+--------+



In [14]:
payment_selected = payment_selected.withColumn(
    "Is_Entity",
    when(payment_selected["Third_Party_Payment_Recipient_Indicator"] == "Entity", 1).otherwise(0)
).withColumn(
    "Is_Individual",
    when(payment_selected["Third_Party_Payment_Recipient_Indicator"] == "Individual", 1).otherwise(0)
)

In [ ]:
payment_selected.groupBy("Charity_Indicator") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+----------------------+-------+
|Charity_Indicator     |count  |
+----------------------+-------+
|NULL                  |9119161|
|No                    |4166622|
|Yes                   |664    |
|No Third Party Payment|2      |
+----------------------+-------+



In [15]:
payment_selected = payment_selected.withColumn(
    "Is_Charity",
    when(payment_selected["Charity_Indicator"] == "Yes", 1).otherwise(0)
).withColumn(
    "Is_Physician_Owned",
    when(payment_selected["Physician_Ownership_Indicator"] == "Yes", 1).otherwise(0)
)

In [ ]:
payment_selected.groupBy("Is_Charity") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+----------+--------+
|Is_Charity|count   |
+----------+--------+
|0         |13285785|
|1         |664     |
+----------+--------+



In [16]:
grouped_df = payment_selected.groupBy("Covered_Recipient_NPI").agg(
    sum("Total_Amount_of_Payment_USDollars").alias("Total_Payment_Sum"),
    sum("Number_of_Payments_Included_in_Total_Amount").alias("Num_Payments_Sum"),
    max("Is_Physician_Owned").alias("Is_Physician_Owned"),
    max("Is_Charity").alias("Is_Charity"),
    max("Is_Entity").alias("Is_Entity"),
    max("Is_Individual").alias("Is_Individual"),
    max("Is_Covered").alias("Is_Covered")
)

grouped_df.show(truncate=False)

+---------------------+------------------+----------------+------------------+----------+---------+-------------+----------+
|Covered_Recipient_NPI|Total_Payment_Sum |Num_Payments_Sum|Is_Physician_Owned|Is_Charity|Is_Entity|Is_Individual|Is_Covered|
+---------------------+------------------+----------------+------------------+----------+---------+-------------+----------+
|1629271788           |17.24             |1.0             |0                 |0         |0        |0            |1         |
|1154818698           |218.3             |8.0             |0                 |0         |0        |0            |1         |
|1952358087           |9515.409999999998 |195.0           |0                 |0         |0        |0            |1         |
|1366702532           |419.98            |7.0             |0                 |0         |0        |0            |1         |
|1376701391           |4805.67           |40.0            |0                 |0         |0        |0            |1         |


In [ ]:
grouped_df.groupBy("Is_Charity") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+----------+------+
|Is_Charity|count |
+----------+------+
|0         |859513|
|1         |296   |
+----------+------+



## Combine into one final dataset

In [17]:
opioids = pd.read_csv('/content/drive/MyDrive/ISYE 6740 Project Files/opioids_cleaned.csv')

selected_cols = [
    "Prscrbr_NPI", "Prscrbr_Last_Org_Name", "Prscrbr_First_Name", "Prscrbr_State_Abrvtn","Prscrbr_Type",
    "Gnrc_Name", "Tot_Clms", "Tot_30day_Fills", "Tot_Day_Suply", "Tot_Drug_Cst"
]

claims_selected = claims.select(*selected_cols)

opioids_spark = spark.createDataFrame(opioids)

claims_selected = claims_selected.withColumn("Gnrc_Name", upper(claims_selected["Gnrc_Name"]))

joined_df = claims_selected.join(opioids_spark, claims_selected["Gnrc_Name"] == opioids_spark["Generic Name"], how="left")

group_cols = ["Prscrbr_NPI", "Prscrbr_Last_Org_Name", "Prscrbr_First_Name", "Prscrbr_State_Abrvtn", "Prscrbr_Type"]

# List of numeric columns to aggregate
agg_cols = ["Long-acting opioid", "Tot_Clms", "Tot_30day_Fills", "Tot_Day_Suply", "Tot_Drug_Cst"]

# Perform grouping and summation
result = joined_df.groupBy(*group_cols).sum(*agg_cols)
result = result.select(
    *group_cols,
    result["sum(Long-acting opioid)"].alias("Long_acting_opioid"),
    result["sum(Tot_Clms)"].alias("Total_Claims"),
    result["sum(Tot_30day_Fills)"].alias("Total_30day_Fills"),
    result["sum(Tot_Day_Suply)"].alias("Total_Day_Supply"),
    result["sum(Tot_Drug_Cst)"].alias("Total_Drug_Cost")
)

result_2 = result.join(grouped_df, result["Prscrbr_NPI"] == grouped_df["Covered_Recipient_NPI"], how="inner")

In [20]:
leie = pd.read_csv('/content/drive/MyDrive/ISYE 6740 Project Files/leie_cleaned.csv')
leie_filtered = leie[leie['NPI'].notnull()]

fraud_npis = set(leie_filtered['NPI'].astype(int))  # cast to string to match Spark format if needed

<ipython-input-20-b20f8202d3e6>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  leie = pd.read_csv('/content/drive/MyDrive/ISYE 6740 Project Files/leie_cleaned.csv')


In [19]:
fraud_npis

{1265696769,
 1629323266,
 1740210178,
 1700986890,
 1508900879,
 1841692688,
 1932296217,
 1457684508,
 1023115292,
 1457782815,
 1568735264,
 1346207776,
 1467678755,
 1912373283,
 1356660781,
 1689649212,
 1326481474,
 1871183947,
 1619525714,
 1912963174,
 1285685354,
 1265631345,
 1871577213,
 1982693511,
 1851392138,
 1811218571,
 1790935179,
 1760723092,
 1679589535,
 1720189087,
 1497006241,
 1942290598,
 1902149800,
 1770553521,
 1710063797,
 1164083382,
 1942618293,
 1699938498,
 1992753347,
 1053458638,
 1730379983,
 1205272788,
 1336246487,
 1942356183,
 1821671645,
 1841594595,
 1245282532,
 1083703532,
 1245217017,
 1306919162,
 1568669950,
 1902903552,
 1134100738,
 1477673218,
 1396736260,
 1831403781,
 1245413640,
 1124106513,
 1174733075,
 1427079458,
 1053360420,
 1558479147,
 1912439084,
 1982791984,
 1477738805,
 1164443958,
 1992720700,
 1780842815,
 1912471871,
 1699086660,
 1639121221,
 1558315341,
 1912013137,
 1922138452,
 1700266335,
 1548386659,
 1801814371,

In [21]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

final_result = result_2.withColumn("Fraud",
    when(col("Prscrbr_NPI").isin(fraud_npis), 1).otherwise(0)
)

print("Final number of rows:", final_result.count())

Final number of rows: 569166


In [ ]:
final_result.groupBy("Fraud") \
    .count() \
    .orderBy("count", ascending=False) \
    .show()

+-----+------+
|Fraud| count|
+-----+------+
|    0|569033|
|    1|   133|
+-----+------+



In [22]:
final_result.printSchema()

root
 |-- Prscrbr_NPI: integer (nullable = true)
 |-- Prscrbr_Last_Org_Name: string (nullable = true)
 |-- Prscrbr_First_Name: string (nullable = true)
 |-- Prscrbr_State_Abrvtn: string (nullable = true)
 |-- Prscrbr_Type: string (nullable = true)
 |-- Long_acting_opioid: long (nullable = true)
 |-- Total_Claims: long (nullable = true)
 |-- Total_30day_Fills: double (nullable = true)
 |-- Total_Day_Supply: long (nullable = true)
 |-- Total_Drug_Cost: double (nullable = true)
 |-- Covered_Recipient_NPI: string (nullable = true)
 |-- Total_Payment_Sum: double (nullable = true)
 |-- Num_Payments_Sum: double (nullable = true)
 |-- Is_Physician_Owned: integer (nullable = true)
 |-- Is_Charity: integer (nullable = true)
 |-- Is_Entity: integer (nullable = true)
 |-- Is_Individual: integer (nullable = true)
 |-- Is_Covered: integer (nullable = true)
 |-- Fraud: integer (nullable = false)



In [ ]:
# Get non-null counts
non_null_counts = final_result.select([count(col(c)).alias(c) for c in final_result.columns])
non_null_dict = non_null_counts.collect()[0].asDict()

# Get distinct counts
distinct_counts = final_result.select([countDistinct(col(c)).alias(c) for c in final_result.columns])
distinct_dict = distinct_counts.collect()[0].asDict()

# Combine both into a list of tuples
combined_data = [
    (col_name, non_null_dict.get(col_name, 0), distinct_dict.get(col_name, 0))
    for col_name in final_result.columns
]

# Create final DataFrame
transpose_df = spark.createDataFrame(
    combined_data,
    ["Column", "Non-Null Count", "Distinct Count"]
)

transpose_df.show(50, truncate=False)
print("Total number of rows in the dataset:", final_result.count())

+---------------------+--------------+--------------+
|Column               |Non-Null Count|Distinct Count|
+---------------------+--------------+--------------+
|Prscrbr_NPI          |569166        |569166        |
|Prscrbr_Last_Org_Name|569166        |160420        |
|Prscrbr_First_Name   |569166        |52015         |
|Prscrbr_State_Abrvtn |569166        |61            |
|Long_acting_opioid   |230581        |48            |
|Total_Claims         |569166        |22475         |
|Total_30day_Fills    |569166        |177274        |
|Total_Day_Supply     |569166        |192976        |
|Total_Drug_Cost      |569166        |520624        |
|Covered_Recipient_NPI|569166        |569166        |
|Total_Payment_Sum    |569166        |243995        |
|Num_Payments_Sum     |569166        |568           |
|Is_Physician_Owned   |569166        |2             |
|Is_Charity           |569166        |2             |
|Is_Entity            |569166        |2             |
|Is_Individual        |56916

In [23]:
final_result.coalesce(1).write.mode("overwrite").csv('/content/drive/MyDrive/ISYE 6740 Project Files/data_cleaned.csv', header=True)